In [1]:
import math
import numpy as np
from stl import mesh
import pandas as pd

In [2]:
# Read in mesh
test_vessel = list(mesh.Mesh.from_file("testvessel.stl"))
# Process mesh so that it is in the form of points[] and connectivity[]

# Create list of points, represented as a DataFrame
# points_list = pd.DataFrame(index = range(1), columns = range(3))
# connectivity_list = pd.DataFrame(index = range(len(test_vessel), columns = range(3)))
points_list = []
connectivity_list = []
for triangle in test_vessel:
        pt1 = list(triangle[0:3]) # [x, y, z]
        pt2 = list(triangle[3:6])
        pt3 = list(triangle[6:9])
        if pt1 not in points_list:
                points_list.append(pt1)
        if pt2 not in points_list:
                points_list.append(pt2)
        if pt3 not in points_list:
                points_list.append(pt3)

        connectivity_list.append([points_list.index(pt1), points_list.index(pt2), points_list.index(pt3)])
print(len(points_list), len(connectivity_list))

5919 11838


In [3]:
# Bounding box

x_max = -1 * float("inf")
x_min = float("inf")
y_max = -1 * float("inf")
y_min = float("inf")
z_max = -1 * float("inf")
z_min = float("inf")

for i in range(0, len(points_list)):
        x = points_list[i][0]
        y = points_list[i][1]
        z = points_list[i][2]
        x_max = max(x, x_max)
        x_min = min(x, x_min)
        y_max = max(y, y_max)
        y_min = min(y, y_min)
        z_max = max(z, z_max)
        z_min = min(z, z_min)

points_rows = len(points_list) - 1
# Create coordinates of bounding box
box_points = [[x_min, y_min, z_min], 
[x_min, y_min, z_max], 
[x_min, y_max, z_min], 
[x_min, y_max, z_max], 
[x_max, y_min, z_min],
[x_max, y_min, z_max], 
[x_max, y_max, z_min], 
[x_max, y_max, z_max]]
for i in range(0, len(box_points)):
        points_list.append(box_points[i])

box_connectivity = [[5 + points_rows, 1 + points_rows, 6 + points_rows], 
[2 + points_rows, 1 + points_rows, 6 + points_rows], 
[4 + points_rows, 2 + points_rows, 6 + points_rows],
[6 + points_rows, 4 + points_rows, 8 + points_rows],
[6 + points_rows, 5 + points_rows, 7 + points_rows],
[7 + points_rows, 6 + points_rows, 8 + points_rows],
[3 + points_rows, 1 + points_rows, 5 + points_rows], 
[5 + points_rows, 3 + points_rows, 7 + points_rows],
[2 + points_rows, 1 + points_rows, 3 + points_rows],
[3 + points_rows, 2 + points_rows, 4 + points_rows],
[4 + points_rows, 3 + points_rows, 8 + points_rows],
[7 + points_rows, 3 + points_rows, 8 + points_rows]]
for i in range(0, len(box_connectivity)):
        connectivity_list.append(box_connectivity[i])


# Write new stl file for bounding box

# case points and connectivity to numpy arrays
points_arr = np.array(points_list)
connectivity_arr = np.array(connectivity_list)

with_box = mesh.Mesh(np.zeros(connectivity_arr.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(connectivity_arr):
        # print(f)
        for j in range(3):
                # print(f[j])
                with_box.vectors[i][j] = points_arr[f[j], :]
with_box.save("withboxpython.stl")